<a href="https://colab.research.google.com/github/AliSheheryar/Mystic00/blob/master/Iris_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving Iris.csv to Iris.csv


In [ ]:
import numpy as np
import pandas as pd
import io

Iris_df = pd.read_csv(io.BytesIO(uploaded['Iris.csv']))
Iris_df.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [ ]:
# After Importing the data into Pandas, numpy conv -> shuffle 
Iris_df_raw = Iris_df.iloc[ : , 1:]
Iris_np = np.array(Iris_df_raw)

Iris_np = np.random.permutation(Iris_np)
Iris_np[:9] 


array([[6.7, 3.0, 5.0, 1.7, 'Iris-versicolor'],
       [6.2, 2.8, 4.8, 1.8, 'Iris-virginica'],
       [6.0, 3.0, 4.8, 1.8, 'Iris-virginica'],
       [4.6, 3.6, 1.0, 0.2, 'Iris-setosa'],
       [6.5, 3.0, 5.2, 2.0, 'Iris-virginica'],
       [6.8, 3.2, 5.9, 2.3, 'Iris-virginica'],
       [5.1, 2.5, 3.0, 1.1, 'Iris-versicolor'],
       [4.8, 3.1, 1.6, 0.2, 'Iris-setosa'],
       [6.7, 3.3, 5.7, 2.1, 'Iris-virginica']], dtype=object)

In [ ]:
# Input data gets scaled while labels get OneHotEncoded 

from sklearn import preprocessing as pp
from tensorflow.keras.utils import to_categorical

minmax_scaler = pp.MinMaxScaler()



input_data = Iris_np[ : , :-1]     #select all rows,all columns except last Col
label_data = Iris_np[ : , -1:]   # Select all rows, last col. only 


print(input_data[:4])
print(" ___    ____ ____")
print(label_data[:4])

#Skinning the input data using scikit 

scaled_input = minmax_scaler.fit_transform(input_data) #Scaled X
label_data_Pandas = pd.DataFrame(label_data)
label_data_Pandas = pd.get_dummies(label_data_Pandas)
label_data_np = np.array(label_data_Pandas)            #OneHotEncoded Y

print(" Scaled Cat")
print(scaled_input[:4])
print("_______")
print(label_data_np[ :4])

[[6.7 3.0 5.0 1.7]
 [6.2 2.8 4.8 1.8]
 [6.0 3.0 4.8 1.8]
 [4.6 3.6 1.0 0.2]]
 ___    ____ ____
[['Iris-versicolor']
 ['Iris-virginica']
 ['Iris-virginica']
 ['Iris-setosa']]
 Scaled Cat
[[0.66666667 0.41666667 0.6779661  0.66666667]
 [0.52777778 0.33333333 0.6440678  0.70833333]
 [0.47222222 0.41666667 0.6440678  0.70833333]
 [0.08333333 0.66666667 0.         0.04166667]]
_______
[[0 1 0]
 [0 0 1]
 [0 0 1]
 [1 0 0]]


In [ ]:
# Split data into test and Training/Val data gangs and let the Party Begin

#concat the arays (data and labels) for the Ultimate Split

#   ***** Yeee HaaaaaaaaaaaaaWWWWWWWWWWWWWWWWwwwwwwwwWWWWWWW********

glued_data = np.concatenate((scaled_input, label_data_np) , axis =1)

#Splitting into test/train_val data chunks 

test_data = glued_data[120 : ]         # 120 -149
train_val_data = glued_data[:120]           #0 -119   

print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
#print(glued_data[119 : 122])          #119,120,121

print( " *****##############************** ")

#print(test_data[ :2])                 #120 ,121

#print("***** ______^^^^^^^^^^^^^^^^^^^^^^*****")
print(train_val_data[ -3: ])              #117 ,118,119

print(type(train_val_data))
print(train_val_data.shape)




^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 *****##############************** 
[[0.66666667 0.54166667 0.79661017 1.         0.         0.
  1.        ]
 [0.38888889 0.33333333 0.59322034 0.5        0.         1.
  0.        ]
 [0.58333333 0.29166667 0.72881356 0.75       0.         0.
  1.        ]]
<class 'numpy.ndarray'>
(120, 7)


In [ ]:
# Define a Sequential model to train on given data
from keras import models
from keras import layers


def build_model(X_train) :

  model = models.Sequential()
  model.add(layers.Dense(64 , activation = 'relu' , input_shape = (X_train.shape[1] , ))) #Input Layer

  model.add(layers.Dense(32 , activation = 'relu' ))
  model.add(layers.Dense(16 , activation = 'relu'))

  model.add(layers.Dense(3 , activation = 'softmax'))  #Output Layer

  model.compile(optimizer = 'rmsprop' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

  return model

  




In [ ]:
# Data(X) and Target(Y) spliter function
#for IRIS classifier task last three columns are Y(label)
#remaining columns are X(input)

def splitter(packed_array):
  print(packed_array.shape)
  X = packed_array[ : , :-3]
  Y = packed_array[ : , -3: ]

  return X,Y

  
"""
a,b = splitter(train_val_data)
print(a.shape)
print(b.shape)
"""

'\na,b = splitter(train_val_data)\nprint(a.shape)\nprint(b.shape)\n'

In [ ]:
# Applying the K-fold validation to obtain high accuracy prediction model 

"""
     set chunk size and cntr variables and copy slices into 
     pavaillions and start jamming with a fit method impregrnated for loop

     uneven datasets should be split too  - > conmdiotins         
"""  
"""
At the end of every iteration, Manhattan distance will be calulated 
to check if distance (d):   d < chunk_size

If so, put the leftover to val_set and break the loop
"""

def K_fold_Validation( inp_arr , k_val):


  chunk_size = len(inp_arr) // k_val  # 120//4 = 30
  d=0     #Manhattan Distance 
  start = stop = 0
  acc_score = list()
  print("Inp_shape , chunk size , k-value =" ,inp_arr.shape ,chunk_size,k_val)
  

# Keep in mind, we are dealing with 2D Numpy array [120, 7] shape. 
  
  for i in range(k_val):

  
    start = i*chunk_size     
    stop  = start + chunk_size

    val_data = inp_arr[start : stop]
    print("val data shape :" , val_data.shape) #So far , so good*

    train_data = np.concatenate([inp_arr[ :start] , inp_arr[stop : ] ] , axis = 0)

    """
    ttt = np.isin(inp_arr , val_data)  # Disputedd*****************
    pre_train = np.logical_not(ttt)

    #pre_train = pre_train.reshape(90,7)   ------
    
    train_data_0 = inp_arr[pre_train]
    train_data = train_data_0.reshape(120,7)
    """

    X_train , Y_train = splitter(train_data)
    X_valid , Y_valid = splitter(val_data)

    print("Before **")
    model = build_model(X_train)
    history = model.fit( X_train,Y_train, epochs = 40 , batch_size = 32)
    losss , accuracy = model.evaluate(X_valid , Y_valid )
 
    acc_score.append(accuracy)

# code block below checks for chunk_size not completely divisible
# if the last val_segment is uneven, it takes whatever remains as val_data
# and remaing as train data for last k-fold iteration
    d = len(inp_arr) - stop
    if d < chunk_size:
      val_data = inp_arr[stop : ]
      train_data = inp_arr[ : stop]

      X_train , Y_train = splitter(train_data)
      X_val , Y_val = splitter(val_data)

      history = model.fit(X_train , Y_train , epochs =40 , batch_size = 32)
      loss, accuracy = model.evaluate(X_valid , Y_valid)
      acc_score.append(accuracy)
      print("Within **")

      break
  print(history.history.keys())
  
  return acc_score




In [ ]:
# Calling k-Fold validation on the training/set and see the averge 
# score of the model

# Pre-Tune Phase 




score = K_fold_Validation(train_val_data , 4)  #k=4, 4-fold validation on input data 
m =  np.mean(score)
print(m)


In [ ]:
print(history.history.keys())

In [ ]:
 import numpy as np
 x = np.arange(12).reshape(4,3)
 y = np.arange(6).reshape(2,3)
 print(x)
 print(y)

 ki = np.isin(x,y)
 print(ki)
 print("********************")
 print(x.size)
 #print(x[ki].reshape(x.size - y.size,3))

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]]
[[0 1 2]
 [3 4 5]]
[[ True  True  True]
 [ True  True  True]
 [False False False]
 [False False False]]
********************
12
